### Tìm phòng trọ

In [ ]:
from typing import List, Dict, Tuple

# =========================
# 1. DỮ LIỆU 10 SINH VIÊN
# =========================

Student = Dict[str, object]

students: List[Student] = [
    {"id": "S1",  "gender": "F", "price": 3000000, "area": "Cau Giay",   "pet": True,  "sleep": 23},
    {"id": "S2",  "gender": "F", "price": 3200000, "area": "Cau Giay",   "pet": False, "sleep": 22},
    {"id": "S3",  "gender": "F", "price": 2800000, "area": "Nam Tu Liem","pet": True,  "sleep": 23},
    {"id": "S4",  "gender": "M", "price": 2500000, "area": "Cau Giay",   "pet": False, "sleep": 22},
    {"id": "S5",  "gender": "M", "price": 2700000, "area": "Bac Tu Liem","pet": True,  "sleep": 0},
    {"id": "S6",  "gender": "F", "price": 3500000, "area": "Cau Giay",   "pet": False, "sleep": 22},
    {"id": "S7",  "gender": "M", "price": 2900000, "area": "Nam Tu Liem","pet": True,  "sleep": 1},
    {"id": "S8",  "gender": "F", "price": 2600000, "area": "Cau Giay",   "pet": True,  "sleep": 23},
    {"id": "S9",  "gender": "M", "price": 2400000, "area": "Cau Giay",   "pet": False, "sleep": 22},
    {"id": "S10", "gender": "F", "price": 3100000, "area": "Nam Tu Liem","pet": True,  "sleep": 0},
]

# ====================================================
# 2. HÀM TÍNH "KHOẢNG CÁCH KHÔNG TƯƠNG THÍCH" GIỮA 2 SV
#    Dùng cho Prim/Kruskal: trọng số càng nhỏ => càng hợp
# ====================================================

def compatibility_distance(a: Student, b: Student) -> float:
    penalty = 0.0

    # Khác giới: phạt rất lớn để ưu tiên ghép cùng giới
    if a["gender"] != b["gender"]:
        penalty += 100.0

    # Chênh lệch tiền: càng lệch phạt càng nhiều
    penalty += abs(a["price"] - b["price"]) / 500000.0

    # Khác khu vực: phạt
    if a["area"] != b["area"]:
        penalty += 2.0

    # Thói quen thú cưng khác: phạt nhẹ
    if a["pet"] != b["pet"]:
        penalty += 1.0

    # Chênh lệch giờ ngủ: phạt
    penalty += abs(a["sleep"] - b["sleep"]) / 3.0

    return penalty

# =========================================
# 3. XÂY TẬP CẠNH VÀ THUẬT TOÁN KRUSKAL (MST)
# =========================================

def build_edges(nodes: List[Student]) -> List[Tuple[float, int, int]]:
    edges: List[Tuple[float, int, int]] = []
    n = len(nodes)
    for i in range(n):
        for j in range(i + 1, n):
            w = compatibility_distance(nodes[i], nodes[j])
            edges.append((w, i, j))
    return edges

def kruskal_mst(nodes: List[Student]) -> List[Tuple[float, int, int]]:
    edges = build_edges(nodes)
    edges.sort(key=lambda x: x[0])  # Greedy: luôn chọn cạnh nhẹ nhất trước

    parent = list(range(len(nodes)))
    rank = [0] * len(nodes)

    def find(x: int) -> int:
        if parent[x] != x:
            parent[x] = find(parent[x])
        return parent[x]

    def union(x: int, y: int) -> bool:
        rx, ry = find(x), find(y)
        if rx == ry:
            return False
        if rank[rx] < rank[ry]:
            parent[rx] = ry
        elif rank[rx] > rank[ry]:
            parent[ry] = rx
        else:
            parent[ry] = rx
            rank[rx] += 1
        return True

    mst: List[Tuple[float, int, int]] = []
    for w, i, j in edges:
        if union(i, j):
            mst.append((w, i, j))
            if len(mst) == len(nodes) - 1:
                break

    return mst

# ========================================================
# 4. GREEDY: XẾP HẠNG ĐỘ PHÙ HỢP CỦA 10 SV VỚI SINH VIÊN MỚI
#    Input: giới tính, max_price, khu vực
#    Output: danh sách từ phù hợp nhất -> kém nhất
# ========================================================

def score_for_user(target: Student, candidate: Student) -> float:
    # Nếu yêu cầu cùng giới mà khác => đẩy xuống dưới
    if candidate["gender"] != target["gender"]:
        return -1e9

    score = 0.0

    # Giá: ưu tiên <= max_price và càng gần càng tốt
    if candidate["price"] <= target["price"]:
        score += 3.0
        score += (target["price"] - candidate["price"]) / max(1.0, float(target["price"])) * 2.0
    else:
        # Vượt budget: trừ mạnh
        score -= 3.0
        score -= (candidate["price"] - target["price"]) / 500000.0

    # Khu vực trùng: cộng điểm
    if candidate["area"] == target["area"]:
        score += 3.0

    # Có thể cộng thêm theo pet, sleep nếu muốn:
    # Ví dụ: giống pet:
    # if "pet" in target and candidate["pet"] == target["pet"]:
    #     score += 1.0

    return score

def rank_candidates(target: Student, students: List[Student]) -> List[Dict[str, object]]:
    ranked = []
    for s in students:
        sc = score_for_user(target, s)
        ranked.append({
            "id": s["id"],
            "gender": s["gender"],
            "price": s["price"],
            "area": s["area"],
            "score": sc
        })
    # Sắp xếp giảm dần theo điểm phù hợp
    ranked.sort(key=lambda x: x["score"], reverse=True)
    return ranked

# =========================
# 5. CHƯƠNG TRÌNH CHÍNH
# =========================

def main():
    print("=== TƯ VẤN BẠN Ở GHÉP (Prim/Kruskal + Greedy) ===")
    gender_input = input("Nhập giới tính của bạn (M/F): ").strip().upper()
    max_price_input = int(input("Nhập mức giá mong muốn tối đa (VND): ").strip())
    area_input = input("Nhập khu vực mong muốn (vd: Cau Giay, Nam Tu Liem): ").strip()

    # Sinh viên mới coi như 1 node trong đồ thị
    target: Student = {
        "id": "USER",
        "gender": gender_input,
        "price": max_price_input,
        "area": area_input,
        # giả sử mặc định, có thể cho input thêm
        "pet": True,
        "sleep": 23,
    }

    # Ghép USER vào danh sách để xây MST bằng Kruskal
    all_nodes = students + [target]
    mst = kruskal_mst(all_nodes)

    # Xếp hạng các sinh viên hiện có theo độ phù hợp (Greedy trên tiêu chí)
    ranking = rank_candidates(target, students)

    print("\nBẢNG XẾP HẠNG ĐỘ PHÙ HỢP:")
    print(f"{'ID':<5} {'Gender':<7} {'Price':<10} {'Area':<12} {'Score':>8}")
    for r in ranking:
        print(f"{r['id']:<5} {r['gender']:<7} {r['price']:<10} {r['area']:<12} {r['score']:>8.2f}")

    # Nếu cần minh họa MST: các cạnh kết nối USER với những bạn phù hợp nhất trong cây khung
    print("\nCÁC CẠNH TRONG MST CÓ LIÊN QUAN TỚI USER (càng nhẹ càng hợp):")
    user_index = len(all_nodes) - 1
    for w, i, j in mst:
        if i == user_index or j == user_index:
            other = j if i == user_index else i
            print(f"USER - {all_nodes[other]['id']}: weight = {w:.2f}")

if __name__ == "__main__":
    main()


### Tìm bạn ở ghép

In [2]:
from typing import List, Dict, Tuple

# =========================
# 1. KHAI BÁO KIỂU VÀ DỮ LIỆU
# =========================

Student = Dict[str, object]

# Đây là ví dụ 10 sinh viên.
# Khi làm bài, có thể nhập từ file hoặc từ input ngoài.
students: List[Student] = [
    {"id": "S1",  "gender": "F", "price": 3000000, "area": "Cau Giay",    "pet": False, "sleep": 23},
    {"id": "S2",  "gender": "F", "price": 3200000, "area": "Cau Giay",    "pet": False, "sleep": 22},
    {"id": "S3",  "gender": "F", "price": 2800000, "area": "Nam Tu Liem", "pet": True,  "sleep": 23},
    {"id": "S4",  "gender": "M", "price": 2500000, "area": "Cau Giay",    "pet": False, "sleep": 22},
    {"id": "S5",  "gender": "M", "price": 2700000, "area": "Bac Tu Liem", "pet": True,  "sleep": 0},
    {"id": "S6",  "gender": "F", "price": 3500000, "area": "Cau Giay",    "pet": False, "sleep": 22},
    {"id": "S7",  "gender": "M", "price": 2900000, "area": "Nam Tu Liem", "pet": True,  "sleep": 1},
    {"id": "S8",  "gender": "F", "price": 2600000, "area": "Cau Giay",    "pet": True,  "sleep": 23},
    {"id": "S9",  "gender": "M", "price": 2400000, "area": "Cau Giay",    "pet": False, "sleep": 22},
    {"id": "S10", "gender": "F", "price": 3100000, "area": "Nam Tu Liem", "pet": True,  "sleep": 0},
]

# ==========================================
# 2. CHUẨN HÓA VÀ HÀM TRỌNG SỐ w(u, v)
#    w(u,v) = độ không phù hợp (càng nhỏ càng hợp)
# ==========================================

def compute_normalization_stats(students: List[Student]) -> Dict[str, float]:
    prices = [s["price"] for s in students]
    sleeps = [s["sleep"] for s in students]

    price_range = max(prices) - min(prices)
    sleep_range = max(sleeps) - min(sleeps)

    # Tránh chia cho 0
    if price_range == 0:
        price_range = 1
    if sleep_range == 0:
        sleep_range = 1

    return {
        "price_range": float(price_range),
        "sleep_range": float(sleep_range),
    }

def weight(u: Student, v: Student, stats: Dict[str, float]) -> float:
    """
    Hàm trọng số w(u,v) chuẩn hóa:
    - Khác giới: cộng 1.0
    - Chênh lệch giá: diff / price_range
    - Khác khu vực: cộng 0.6
    - Khác thói quen nuôi pet: cộng 0.4
    - Chênh lệch giờ ngủ: diff / sleep_range
    Tổng càng nhỏ thì càng hợp để ở ghép.
    """

    w = 0.0

    # Giới tính
    if u["gender"] != v["gender"]:
        w += 1.0

    # Giá tiền
    w += abs(u["price"] - v["price"]) / stats["price_range"]

    # Khu vực
    if u["area"] != v["area"]:
        w += 0.6

    # Nuôi thú cưng
    if u["pet"] != v["pet"]:
        w += 0.4

    # Giờ đi ngủ
    w += abs(u["sleep"] - v["sleep"]) / stats["sleep_range"]

    return w

# ==========================================
# 3. TẠO CẠNH VÀ KRUSKAL MST
# ==========================================

def build_edges(students: List[Student], stats: Dict[str, float]) -> List[Tuple[float, int, int]]:
    edges: List[Tuple[float, int, int]] = []
    n = len(students)
    for i in range(n):
        for j in range(i + 1, n):
            w_ij = weight(students[i], students[j], stats)
            edges.append((w_ij, i, j))
    return edges

def kruskal_mst(students: List[Student]) -> List[Tuple[float, int, int]]:
    n = len(students)
    stats = compute_normalization_stats(students)
    edges = build_edges(students, stats)
    edges.sort(key=lambda x: x[0])  # Greedy: luôn chọn cạnh nhẹ nhất trước

    parent = list(range(n))
    rank = [0] * n

    def find(x: int) -> int:
        while parent[x] != x:
            parent[x] = parent[parent[x]]
            x = parent[x]
        return x

    def union(x: int, y: int) -> bool:
        rx, ry = find(x), find(y)
        if rx == ry:
            return False
        if rank[rx] < rank[ry]:
            parent[rx] = ry
        elif rank[rx] > rank[ry]:
            parent[ry] = rx
        else:
            parent[ry] = rx
            rank[rx] += 1
        return True

    mst: List[Tuple[float, int, int]] = []
    for w_ij, i, j in edges:
        if union(i, j):
            mst.append((w_ij, i, j))
            if len(mst) == n - 1:
                break

    return mst

# ==========================================
# 4. SUY RA NHÓM ROOMMATE TỪ MST
#    Quy tắc:
#    - Tính ngưỡng T = trung bình trọng số trên MST.
#    - Chỉ giữ các cạnh có w <= T.
#    - Các thành phần liên thông còn lại là nhóm roommate.
# ==========================================

def groups_from_mst(mst: List[Tuple[float, int, int]], num_nodes: int) -> List[List[int]]:
    if not mst:
        return [[i] for i in range(num_nodes)]

    avg_w = sum(w for w, _, _ in mst) / len(mst)

    parent = list(range(num_nodes))

    def find(x: int) -> int:
        while parent[x] != x:
            parent[x] = parent[parent[x]]
            x = parent[x]
        return x

    def union(x: int, y: int):
        rx, ry = find(x), find(y)
        if rx != ry:
            parent[ry] = rx

    # Chỉ nối những cạnh "hợp" (trọng số thấp hơn hoặc bằng trung bình)
    for w, i, j in mst:
        if w <= avg_w:
            union(i, j)

    components = {}
    for i in range(num_nodes):
        r = find(i)
        components.setdefault(r, []).append(i)

    return list(components.values())

# ==========================================
# 5. CHƯƠNG TRÌNH CHẠY DEMO
# ==========================================

def main():
    mst = kruskal_mst(students)

    print("=== CẠNH CỦA CÂY KHUNG NHỎ NHẤT (MST) ===")
    for w_ij, i, j in mst:
        u = students[i]
        v = students[j]
        print(f"{u['id']} - {v['id']} : w = {w_ij:.3f}")

    groups = groups_from_mst(mst, len(students))

    print("\n=== NHÓM ROOMMATE SUY RA TỪ MST ===")
    for idx, comp in enumerate(groups, start=1):
        member_ids = [students[i]["id"] for i in comp]
        print(f"Nhóm {idx}: {member_ids}")

if __name__ == "__main__":
    main()


=== CẠNH CỦA CÂY KHUNG NHỎ NHẤT (MST) ===
S4 - S9 : w = 0.091
S1 - S2 : w = 0.225
S2 - S6 : w = 0.273
S1 - S8 : w = 0.764
S3 - S8 : w = 0.782
S5 - S7 : w = 0.825
S7 - S10 : w = 1.225
S3 - S10 : w = 1.273
S1 - S4 : w = 1.498

=== NHÓM ROOMMATE SUY RA TỪ MST ===
Nhóm 1: ['S1', 'S2', 'S6', 'S8']
Nhóm 2: ['S3']
Nhóm 3: ['S4', 'S9']
Nhóm 4: ['S5']
Nhóm 5: ['S7']
Nhóm 6: ['S10']
